Сейчас будет много копипаста с тетрадки семинара)

In [ ]:
!pip install nltk

In [ ]:
!pip install spacy

In [ ]:
!pip  install pyLDAvis

In [ ]:
import nltk; nltk.download('stopwords')

!python3 -m spacy download en

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [17]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


Чистка даты

In [8]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [9]:
#tokenize

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


программа, которая нам руками будет возвращать N-число самых частых биграмм в тексте

In [10]:
bigrams = [b for l in data_words for b in zip(l[:-1], l[1:])]
freq = nltk.FreqDist(bigrams) #computes freq of occurrence
fdist = freq.keys() # sorted according to freq

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


Mallen, который смог. Будь как маллет.

In [ ]:
!unzip /content/mallet-2.0.8.zip

In [39]:
mallet_path = '/content/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [40]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(4,
  [('gun', 0.027243589743589744),
   ('law', 0.018547008547008546),
   ('people', 0.01769230769230769),
   ('state', 0.01628205128205128),
   ('crime', 0.011175213675213675),
   ('make', 0.010662393162393162),
   ('case', 0.0105982905982906),
   ('weapon', 0.01014957264957265),
   ('kill', 0.00985042735042735),
   ('firearm', 0.008098290598290599)]),
 (2,
  [('ax', 0.18779604346614656),
   ('line', 0.15694781674163116),
   ('max', 0.15133542968594516),
   ('host', 0.1005453170401624),
   ('nntp_poste', 0.04239143414401146),
   ('reply', 0.02579309795804641),
   ('organization', 0.02193209409704255),
   ('ca', 0.018548740198224733),
   ('nntp_posting', 0.008916132627472834),
   ('distribution_usa', 0.008597699319348805)]),
 (5,
  [('people', 0.023467952581876634),
   ('government', 0.020474181233674903),
   ('state', 0.013542294554952783),
   ('country', 0.010689170182841068),
   ('israeli', 0.010387783805505325),
   ('war', 0.009985935302390999),
   ('attack', 0.009021498894916616

## Тренеруем нашу модель

Надо посчитать лучшее количество топиков через функцию. Шаг = 5, потому что коллаб слишком тупой, а на моем компе не работает spacy :D


In [69]:
from tqdm.auto import tqdm

In [76]:
def opt_topics(n, corpus, id2word, data_lemmatized):
  groups_count = []
  for number in tqdm(n):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=number, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
  
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    groups_count.append((coherence_lda, number))
  
  for pair in groups_count:
    print('The number of groups: ', pair[1], ' - ', 'coherence: ', pair[0], '\n', '--------------------')
  
  return sorted(groups_count)[-1][1]

In [77]:
n = [5, 10, 15, 20]
best_topics_count = opt_topics(n, corpus, id2word, data_lemmatized)


The number of groups:  5  -  coherence:  0.4297369063851419 
 --------------------
The number of groups:  10  -  coherence:  0.4973497938960755 
 --------------------
The number of groups:  15  -  coherence:  0.44582805103485196 
 --------------------
The number of groups:  20  -  coherence:  0.4392813747423439 
 --------------------


Видно, что 10 топиков показывают лучший результат. Возможно, между 10 и 15 топиками есть и лучше.


In [79]:
best_topics_count

10

In [80]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=best_topics_count, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [124]:
topics = lda_model.show_topics(num_topics=best_topics_count, num_words=10, formatted=False) #беру show_topics, потому что с print_topics не получается сделать кортежи
doc_lda = lda_model[corpus] #не знаю зачем эта штука, но она была в семе, хуже не будет)

In [125]:
topics[0][1][:10]

[('team', 0.02816584),
 ('year', 0.02723164),
 ('game', 0.025724994),
 ('play', 0.019548114),
 ('win', 0.018154599),
 ('player', 0.014175091),
 ('run', 0.009828578),
 ('last', 0.009554995),
 ('good', 0.0093642585),
 ('hit', 0.008524719)]

In [238]:
def d_sorter(d):
  list_d = list(d.items())
  list_d.sort(key=lambda i: i[1])
  return list_d[-1][0]

In [ ]:
d_sorter(q)

In [232]:
q = {'a': 1, 'b': 2, 'c': 3}

In [240]:
topics_values = {}

#for index, text in enumerate(topics)

for index, text in tqdm(enumerate(data_lemmatized)):
  topic_value = {}
  for topic in topics:
    list_of_topic = topic[1]
    for word in text: #слово в тексте
      for topic in list_of_topic: #кортеж топик-число в списке топиков для этого текста
        if word in topic[0]: 
          try:
            topic_value[word] = topic_value[word] + topic[1]
          except:
            topic_value[word] = topic[1]
  try:
    n_topic = d_sorter(topic_value)
    topics_values[index] = n_topic
  except:
    topics_values[index] = 'n/a'

In [243]:
get_topic_dic(topics)

{'0': {'game': 0.025724994,
  'good': 0.0093642585,
  'hit': 0.008524719,
  'last': 0.009554995,
  'play': 0.019548114,
  'player': 0.014175091,
  'run': 0.009828578,
  'team': 0.02816584,
  'win': 0.018154599,
  'year': 0.02723164},
 '1': {'back': 0.010807149,
  'come': 0.011710706,
  'day': 0.012717305,
  'first': 0.008184588,
  'get': 0.010662511,
  'go': 0.021728566,
  'say': 0.008744546,
  'see': 0.008303262,
  'take': 0.01144027,
  'time': 0.014948809},
 '2': {'also': 0.00569999,
  'cost': 0.007987364,
  'high': 0.0074608973,
  'item': 0.0057826536,
  'large': 0.005205771,
  'low': 0.0067702075,
  'research': 0.006973393,
  'space': 0.014079927,
  'test': 0.0055987528,
  'year': 0.0075525735},
 '3': {'bike': 0.013057647,
  'car': 0.032890327,
  'drive': 0.021455154,
  'engine': 0.008950168,
  'light': 0.0088470625,
  'power': 0.011179995,
  'reality': 0.009504482,
  'slave': 0.0106329555,
  'speed': 0.008960807,
  'wire': 0.01099707},
 '4': {'ax': 0.090422064,
  'di_di': 0.017890

In [110]:
import random